## Interactive Mohr's Circle:

### ENGR 213

##### Why?

One might imagine that graphical tools for solving engineering problems are a thing of the past now that we have access to great computational tools. This could be true. Mohr's Circle is a graphical tool that acts like a 'slide rule' for solving stresses on rotated elements and well as determining the principle planes and stresses in a material. 

### Version 1.0: 6/2/21

There will surely be many errors in this initial version. Perhaps I will have time to catch them and fix them....

### Libraries

There are a number of different widget libraries. In the end the ipywidgets was most adaptable to my purposes. I suspect this would change if I were seeking to build this tool as a webpage. References that I used in sorting this all out are given in my [InteractiveStudy notebook](https://github.com/smithrockmaker/ENGR212/blob/main/InteractiveStudy.ipynbhttps://github.com/smithrockmaker/ENGR212/blob/main/InteractiveStudy.ipynb). At the moment (2/21) this is miserably documented but the references contained therein are much better if they are still live.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, Layout


### Setting Up Global Tools

There may or may not be any items that I can set up globally for this tool...

### Mohr's Circle Function

The development of the construction and plotting of Morh's Circle is documented at some level in the MohrsDev.ipynb notebook. This notebook is an effort to make that tool interactive.

[pyplot.bar documentation](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.bar.html)[


In [6]:
def MohrsCircle(sxx, syy, txy, angle):
    
    # fill the stress tensor
    S = np.array([[sxx, txy],[txy, syy]])
    S11 = S[0,0] # sigma xx
    S12 = S[0,1] # tau xy
    S21 = S[1,0] # - tau xy
    S22 = S[1,1] # sigma yy

    # rotation of stress plane in degrees CCW from horizontal axis
    rotAngleDeg = angle
    rotAngle = (np.pi/180.)*rotAngleDeg
    
    # linear array of angle for plotting - NOT plane angles
    angle = np.linspace( 0 , 2 * np.pi , 150 ) 
    center = [(S11 + S22)/2.0, 0.0]
    radius = np.sqrt((S11 - S22)**2/4.0 + S12**2)
    circleX = center[0] + radius * np.cos( angle ) 
    circleY = center[1] + radius * np.sin( angle ) \

    # Sort out plotting limits
    circleXmin = center[0] - radius
    circleXmax = center[0] + radius

    # where is circle on grid - span y axis, left of y axis, right of y axis

    limScale = 0.6 
    if circleXmin > 0.:
        leftLim = -1.
        rightLim = circleXmax + limScale*radius
    else:
        if circleXmax < 0.:
            rightLim = 1.
            leftLim = circleXmin - limScale*radius
        else:
            leftLim = circleXmin - limScale*radius
            rightLim = circleXmax + limScale*radius

    # End points of global stress line on Mohrs Circle
    pointSize = 20. # set possible size of point on plot
    pointH = [S11, -S21] # right side of stress cube
    pointV = [S22, S12] # top of stress cube
    endX = [pointV[0], pointH[0]]
    endY = [pointV[1], pointH[1]]
    circleVpt = [pointV[0]-center[0], pointV[1]]
    circleHpt = [pointH[0]-center[0], pointH[1]]

    # angles on Mohr's circle
    twoThetaP = np.arctan(pointH[1]/(pointH[0]-center[0]))
    twoThetaPdeg = round((180./np.pi)*twoThetaP,3)
    ThetaPdeg = twoThetaPdeg/2.

    # rotation of stress plane of interest.
    rotatedAngle = twoThetaP + 2*rotAngle
    newH = [center[0] + radius*np.cos(rotatedAngle),radius*np.sin(rotatedAngle)]
    newV = [center[0] + radius*np.cos(rotatedAngle+np.pi),radius*np.sin(rotatedAngle+np.pi)]
    endXnew = [newV[0], newH[0]]
    endYnew = [newV[1], newH[1]]
        
    # develop label strings, have to be regenerated as input values change
    labelCenter = str(round(center[0],3))
    labelNormalY = str(round(pointH[0],3))
    labelNormalX = str(round(pointV[1],3))
    labelInitShear = str(round(pointV[1],3))
    labelInitNx = str(round(pointH[0],3))
    labelInitNy = str(round(pointV[0],3))
    labelMaxShear = str(round(radius,3))
    labelMaxNormal = str(round(center[0]+radius,3))
    labelMinNormal = str(round(center[0]-radius,3))
    labeltwoThetaP = str(round(twoThetaPdeg,2))
    labelThetaP = str(round(ThetaPdeg,2))
    labelRotNx = str(round(newH[0],3))
    labelRotNy = str(round(newV[0],3))
    labelRotShear = str(round(newV[1],3))
    labelRotAngle = str(round(rotAngleDeg,2))
    
    # generate the plots and labels
    fig1, ax1 = plt.subplots()
    ax1.plot(circleX, circleY, color = 'blue', linewidth = 2.5, alpha = 0.3)
    
    # initial stress state points
    ax1.scatter(pointH[0], pointH[1], marker = 'o', s = pointSize, color = 'green')
    ax1.scatter(pointV[0], pointV[1], marker = 'o', s = pointSize, color = 'green')
    ax1.plot(endX, endY, linewidth = 1.,color = 'green')
    
    # rotated plane stress points
    ax1.scatter(newH[0], newH[1], marker = 'o', s = pointSize, color = 'fuchsia')
    ax1.scatter(newV[0], newV[1], marker = 'o', s = pointSize, color = 'fuchsia')
    ax1.plot(endXnew, endYnew, linewidth = 1.,color = 'fuchsia')
    
    # a way to set labels
    plt.xlabel(r"$\sigma$", fontsize = 16)
    plt.ylabel(r"$\tau$", fontsize = 16)
    plt.title('Mohrs Circle', fontsize = 20)
    
    # set limits for good utility
    ax1.set_xlim([leftLim,rightLim])
    ax1.set_aspect('equal') # set aspect ratio so circle looks like a cirle - cool!
    
    # reference lines of various sorts
    # average normal stress (center of circle)
    ax1.vlines(center[0], -1.1*radius, +1.1*radius,
         color = 'black', linestyle = '-',
         linewidth = 1. , label = "average normal stress")
    ax1.hlines(0, center[0]-1.1*radius, center[0]+1.1*radius,
         color = 'black', linestyle = '-',
         linewidth = 1. , label = "0 shear")

    # max shear stress
    ax1.hlines(radius, leftLim, center[0],
         color = 'blue', linestyle = ':',
         linewidth = 1. , label = "max shear stress")
    # max normal stress
    ax1.vlines(center[0]+radius, -1.1*radius, 0,
         color = 'blue', linestyle = ':',
         linewidth = 1. , label = "max normal stress")
    # min normal stress
    ax1.vlines(center[0]-radius,  -1.1*radius, 0,
         color = 'blue', linestyle = ':',
         linewidth = 1. , label = "min normal stress")
    # inital stress state
    ax1.hlines(pointV[1], pointV[0], rightLim,
         color = 'red', linestyle = ':',
         linewidth = 1. , label = "shear stress")
    ax1.hlines(pointH[1], pointH[0], rightLim,
         color = 'red', linestyle = ':',
         linewidth = 1. , label = "shear stress")
    ax1.vlines(pointV[0], pointV[1], 1.2*radius,
         color = 'red', linestyle = ':',
         linewidth = 1. , label = "shear stress")
    ax1.vlines(pointV[0], pointV[1], 1.2*radius,
         color = 'red', linestyle = ':',
         linewidth = 1. , label = "shear stress")
    
    # Labels for various important points.
    # Starting with those points that don't shift
    plt.text(center[0]-(0.15*radius), 1.12*radius, r"$\sigma_{ave} = $"+labelCenter, fontsize= 12.)
    plt.text(leftLim+.1, radius+.1, r"$\tau_{max} = $"+labelMaxShear, fontsize= 12.)
    plt.text(center[0]+radius-(0.15*radius), -1.1*radius, r"$\sigma_{max} = $"+labelMaxNormal, fontsize= 12.)
    plt.text(center[0]-radius-(0.15*radius), -1.1*radius, r"$\sigma_{min} = $"+labelMinNormal, fontsize= 12.)
    # starting stress state
    plt.text(center[0]+radius+(0.05*radius), pointV[1]+(0.05*radius), r"$\tau = $"+labelInitShear, fontsize= 12., color='red')
    plt.text(center[0]+radius+(0.05*radius), pointH[1]+(0.05*radius), r"$\tau = $"+labelInitShear, fontsize= 12., color='red')
    plt.text(pointV[0]-(0.15*radius), 1.2*radius, r"$\sigma_{y} = $"+labelInitNy, fontsize= 12., color='red')
    plt.text(pointH[0]-(0.15*radius), 1.2*radius, r"$\sigma_{x} = $"+labelInitNx, fontsize= 12., color='red')
    
    # principle axes and rotation
    plt.text(center[0]+radius/4, 0.05*radius, r"$\theta_{P} = $"+labelThetaP, fontsize= 12., color='green')
    plt.text(center[0]+S11/3, 0.25*pointH[1], r"$2\theta_{P} = $"+labeltwoThetaP, fontsize= 12., color='green')
    
    # ..moving to those that depend on where V and H are located
    setSm = 0.8
    setLg = 1.1
    plt.text(pointV[0], setSm*pointV[1], "side (V)", fontsize= 10.)
    plt.text(pointH[0], setSm*pointH[1], "top (H)", fontsize= 10.)
    
    # data tables
    lineStep = .08*radius
    plt.text(leftLim+0.1*radius, .3*radius, "Initial Stress State", fontsize= 10., color = "red")
    plt.text(leftLim+0.1*radius, .3*radius - 1*lineStep, r"$\sigma_{x} = $"+labelInitNx, fontsize= 10., color = "red")
    plt.text(leftLim+0.1*radius, .3*radius - 2*lineStep, r"$\sigma_{y} = $"+labelInitNy, fontsize= 10., color = "red")
    plt.text(leftLim+0.1*radius, .3*radius - 3*lineStep, r"$\tau_{xy} = $"+labelInitShear, fontsize= 10., color = "red")
    
    plt.text(circleXmax+0.1*radius, .3*radius, "Rotated Stress", fontsize= 10., color = "fuchsia")
    plt.text(circleXmax+0.1*radius, .3*radius - 1*lineStep, r"$\sigma_{x} = $"+labelRotNx, fontsize= 10., color = "fuchsia")
    plt.text(circleXmax+0.1*radius, .3*radius - 2*lineStep, r"$\sigma_{y} = $"+labelRotNy, fontsize= 10., color = "fuchsia")
    plt.text(circleXmax+0.1*radius, .3*radius - 3*lineStep, r"$\tau_{xy} = $"+labelRotShear, fontsize= 10., color = "fuchsia")
    plt.text(circleXmax+0.1*radius, .3*radius - 4*lineStep, r"$\theta_{rot} = $"+labelRotAngle+r"$^{\circ}$", fontsize= 10., color = "fuchsia")
    
    fig1.set_size_inches(10, 9)
    ax1.grid()
    
    plt.show()

 

### Test Function

It worked so I commented it out. Keep for future potential comparison to MohrsDev.

In [8]:
#MohrsCircle(30.,-60.,50.,30.)

### Setting up widgets and interactivity

Once the active function is defined then we define the interactive widgets which are mostly sliders for visual connection to the bar graph. In hindsight I might have done well to make the sliders vertical so they move in the same direction as the bars but hey .... got to save something for a rainy day.

The cap# variables are strings for labeling the different sections of the slider array. Hbox and VBox are used to  lay out the panel. Last two lines pull the trigger and set up the interactivity.

In [13]:
# Set up widgetsm - captions
cap1 = widgets.Label(value=r'$\sigma_{xx}$')
cap2 = widgets.Label(value=r'$\sigma_{yy}$')
cap3 = widgets.Label(value=r'$\tau_{xy}$')
cap4 = widgets.Label(value='Rotation Angle')


# normal stress sliders
sxx=widgets.FloatText(min=-100, max=100, value=30, description = r'$\sigma_{xx}$',continuous_update=False,
                     layout=Layout(width='60%'))
syy=widgets.FloatText(min=-100, max=100, value=-60, description = r'$\sigma_{yy}$',continuous_update=False,
                     layout=Layout(width='60%'))

# shear stress slider
txy=widgets.FloatText(min=-100, max=100, value=50, description = r'$\tau_{xy}$',continuous_update=False,
                       layout=Layout(width='60%'))

# plane of interest slider
angle=widgets.FloatText(min=-89, max=89, value=30, description = 'rotated plane',continuous_update=False,
                       layout=Layout(width='60%'))

# An HBox lays out its children horizontally, VBox lays them out vertically
col1 = widgets.VBox([sxx, syy, txy, angle])
#col2 = widgets.VBox([cap2, cap5, WF1, cap6, WF2])
#col3 = widgets.VBox([cap3, cap4, KEf, PEgf, PEsf])
panel = widgets.HBox([col1])



out = widgets.interactive_output(MohrsCircle, {'sxx': sxx, 'syy': syy,
                                             'txy': txy, 'angle': angle,
                                             })

display(out, panel)

Output()